In [39]:
from statsbombpy import sb
import pandas as pd, numpy as np

pd.set_option('display.max_columns', None)
  
comp = sb.competitions()
matches = sb.matches(competition_id=43, season_id=3)
events = sb.events(7532)

    

In [40]:
from mplsoccer import Sbopen
import pandas as pd, numpy as np

parser = Sbopen(dataframe=True)        # dataframe=True → ya te devuelve DataFrames
comp = parser.competition()            # todas las competiciones open-data
matches = parser.match(competition_id=43, season_id=3)  # Eredivisie 20/21
events, related, freeze, tactics = parser.event(7532)

In [41]:
team1, team2 = events.team_name.unique()
mask_team1 = (events.type_name == 'Pass') & (events.team_name == team1)


In [ ]:
import datetime
events[events["timestamp"]>=datetime.time(0, 13, 5, 760000)]

In [ ]:
a = events[events["type_name"]=="Clearance"]
a[(a["team_name"]=="Peru")]

In [43]:
df_pass = events.loc[mask_team1, ['x', 'y', 'end_x', 'end_y', 'outcome_name']]
mask_complete = df_pass.outcome_name.isnull()

In [ ]:
from mplsoccer import Pitch, FontManager, Sbopen
from matplotlib import rcParams
import matplotlib.pyplot as plt
# Set up the pitch
pitch = Pitch(pitch_type='statsbomb', pitch_color='#22312b', line_color='#c7d5cc')
fig, ax = pitch.draw(figsize=(16, 11), constrained_layout=True, tight_layout=False)
fig.set_facecolor('#22312b')

# Plot the completed passes
pitch.arrows(df_pass[mask_complete].x, df_pass[mask_complete].y,
             df_pass[mask_complete].end_x, df_pass[mask_complete].end_y, width=2,
             headwidth=10, headlength=10, color='#ad993c', ax=ax, label='completed passes')

# Plot the other passes
pitch.arrows(df_pass[~mask_complete].x, df_pass[~mask_complete].y,
             df_pass[~mask_complete].end_x, df_pass[~mask_complete].end_y, width=2,
             headwidth=6, headlength=5, headaxislength=12,
             color='#ba4f45', ax=ax, label='other passes')

# Set up the legend
ax.legend(facecolor='#22312b', handlelength=5, edgecolor='None', fontsize=20, loc='upper left')

# Set the title
ax_title = ax.set_title(f'{team1} passes vs {team2}', fontsize=30)

In [ ]:
ball

In [ ]:
import pandas as pd, plotly.express as px

###############################################################################
# 1) LABEL con actor / receptor / tipo de acción
###############################################################################
def build_label(row: pd.Series) -> str:
    """
    Returns a string like:
        "Kevin De Bruyne → Erling Haaland (Pass)"
        "Vivianne Miedema (Shot)"
    """
    if row["type_name"] == "Pass":
        recv = row.get("pass_recipient_name", "")
        return f"{row['player_name']} → {recv} (Pass)"
    return f"{row['player_name']} ({row['type_name']})"

events["time_sec"] = (
    pd.to_timedelta(events["timestamp"].astype(str))   # 00:00:02.200000 → Timedelta
      .dt.total_seconds()                              # → 2.2
)


ball = (
    events.dropna(subset=["x", "y", "time_sec"])
          .assign(label=events.apply(build_label, axis=1))
)


# a:
ball['time_bin'] = (ball['time_sec'] * 10).round().astype(int) / 10 

###############################################################################
# 2) PLOT
###############################################################################
fig = px.scatter(
        ball.sort_values('time_bin'),
        x='x', y='y',
        animation_frame='time_bin',
        hover_name='label',               # jugador(es) + acción
        range_x=[-1, 125], range_y=[-1, 85],
        width=900, height=500,
)


fig.update_traces(marker=dict(size=8, color="red", line=dict(width=0)))
fig.update_layout(
    title="Ball trajectory • StatsBomb coordinates (0.1-second timelapse)",
    xaxis_title="Pitch X (m)", yaxis_title="Pitch Y (m)",
    plot_bgcolor="#3d693d", paper_bgcolor="#222",
    font_color="#EEE",
)

fig.show()

In [ ]:
ball[ball["time_bin"]>=16.2]